# 00 - NMAIA AI Racer Systems Test Tool
Author: George Gorospe, george.gorospe@nmaia.net (updated 1/14/2025)

### This notebook is used to test each of the AI Racer's sytems. Read each subsection to find out more about your AI Racer! This type of test tool is useful for testing the functions of the racer and it's software.

## Camera Test
### The AI Racer uses a single camera, this camera is connected to the racer's computer via Camera Serial Interface (CSI), This unique interface is simple, fast and doen't take up USB ports. However, it does require extra drivers. 

### The following code will test the camera system. If successful, then a single image, captured by the camera, will be displayed.

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

# Instantiate the camera
camera = CSICamera(width=224, height=224)
image = camera.read()
camera.running = True

# Creating a GUI for displaying the image
image_widget = ipywidgets.Image(format='jpeg')

# Callback function for each time a new frame is collected
def update_image(change):
    image = change['new']
    # Process the frame and display it
    image_widget.value = bgr8_to_jpeg(image)

# Start the update event observation and display the GUI
camera.observe(update_image, names='value')
display(image_widget)

### You should see an image captured by your camera. 

## Servo Controller Test
### The AI Racer uses a Servo Controller Board to produce special pulse width modulated (PWM) signals to control the steering servo motor and the brushless motor via an electronic speed controller (ESC). The servo controller board is used because the Nvidia Jetson Orin Nano computer can't create these special signals on its own. The following code block initialized the libraries required to produce the PWM signal and produces a signal to control the steering servo of the AI Racer.

### IMPORTANT: place the racer off the ground, all 4 wheels should be in the air.
### IMPORTANT: befor running the next cell, ensure that the propulsion system is on, and that the remote controller is set to AI MODE.

In [ ]:
# Importing required Libaries
import cv2, glob
import ipywidgets
import ipywidgets.widgets as widgets
from ipywidgets import Layout

import traitlets
from IPython.display import display

# Style and Layout
Style = {'description_width': 'initial'}

# Setup of NvidiaRacecar Object: Used to contro the steering and throttle of our AI racer.
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

# Setup of Steering/Throttle graphic user interface (GUI)
# 1. Create the graphic controls
# 2. Connect callback functions and connect them to the controls so they activate when the controls are used
# 3. Create a display for the controls with instructions

###### 1. Creating graphic controls

# Steering Control textbox
# We want to explore the full range of the servo by issuing commands.
steering_control = widgets.FloatText(value=.2, description='car.steering =', step = 0.01, style =  Style,layout=Layout(width='150px', height='30px'))
throttle_control = widgets.FloatText(value=.26, description='car.throttle =', step = 0.01, style =  Style,layout=Layout(width='150px', height='30px'))


###### 2. Connecting callback functions and connecting them to the controls

# This fuction executes when the steering value is changed
def steering_callback(change):
    car.steering = steering_control.value # Issue servo steering command by the computer to the vehicle's steering servo

def throttle_callback(change):
    car.throttle = throttle_control.value # Issue throttle command by the computer to the vehicle's speed controller

# Callback connections (Observers)
steering_control.observe(steering_callback, names='value') # Observes the steering_control widget
throttle_control.observe(throttle_callback, names='value') # Observes the throttle_control widget

###### 3. Display the controls with instructions
instructions_01 = widgets.Label(value=str("INSTRUCTIONS: Use the following controls to test the racer's control system. Slowly increase/decrease the values."))
control_UI = widgets.VBox([instructions_01, steering_control, throttle_control])

display(control_UI)

## Game Controller Test
### The AI Racer has two controllers: a (black) radio controller from Futaba, that sends signals directly to the racer via receiver and a (grey) game controller that sends signals via bluetooth to the racer's computer. This test checks that the game controller is successfully connected to the computer.

In [ ]:
import Gamepad
import time
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout

# Gamepad settings
gamepadType = Gamepad.XboxONE
recordButton = 'A'
joystickSpeed = 'RAS -Y'
joystickSteering = 'LAS -X'
pollInterval = 0.05

# Connection Check, and initialize gamepad object
if Gamepad.available():
    print('Gamepad connected')
    gamepad = gamepadType() # Initialize gamepad object
    # Set Initial state
    global running
    running = True
    speed = 0.0
    steering = 0.0
    recording = False
else:
    print('Please connect your gamepad...')
    while not Gamepad.available():
        time.sleep(1.0)

# Start the background updating
gamepad.startBackgroundUpdates()

# Create some callback functions for single events
def recordButtonPressed():
    global recording
    recording = not recording
    if recording == True: 
        buttonStyle_recording()
    else: 
        buttonStyle_default()
    
def speedAxisMoved(position):
    global speed
    throttle_slider.value = -position   # Inverted

def steeringAxisMoved(position):
    global steering
    steering_slider.value = position # Non-inverted 
    
def buttonStyle_recording():
    rec.description = "RECORDING"
    rec.button_style = 'danger'

def buttonStyle_default():
    rec.description = "waiting"
    rec.button_style = ''

# Linking callback functions to event handlers
gamepad.addButtonPressedHandler(recordButton, recordButtonPressed)
gamepad.addAxisMovedHandler(joystickSpeed, speedAxisMoved)
gamepad.addAxisMovedHandler(joystickSteering, steeringAxisMoved)

# Gamepad graphical user interface
throttle_slider = widgets.FloatSlider(value=0,min=-1,max=1,step= 0.01,description='Throttle:',disabled=False,continuous_update=True,orientation='vertical',readout=True,readout_format='.2f',)
steering_slider = widgets.FloatSlider(value=0,min=-1,max=1.0,step=0.01,description='Steering:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='.2f',)
rec = widgets.ToggleButton(value=False, description='waiting', disabled=False, button_style='',)
controller_gui = widgets.HBox([throttle_slider, steering_slider, rec])
display(controller_gui)